In [1]:
import pandas as pd

# Récuprér les derniers pixels posées
On commence par récupérer tous les derniers changement de pixels.

## On importe le csv

In [2]:
df = pd.read_csv("csv/2022_place_canvas_history.csv", usecols = ["pixel_color", "coordinate", "timestamp"])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

## Premier tri des données
Il faut trier par timestamp et supprimer tout les blancs a la fin

In [3]:
df = df.sort_values(by="timestamp")

In [4]:
df.reset_index(inplace=True, drop=True)

In [5]:
df_white_list = df[df.pixel_color == "#FFFFFF"]

In [6]:
df_white_list_index = df_white_list.index.tolist()

In [7]:
ancien = df_white_list_index[len(df_white_list_index) - 1]
for n in list(reversed(df_white_list_index)):
    if n == ancien:
        ancien = n - 1
    else:
        limit_delete = ancien + 1
        break

In [8]:
df.drop([i for i in range(limit_delete, df.shape[0])], inplace=True)

## On réucpère tout les denriers changements et on exporte

In [9]:
last_pixels = df.groupby("coordinate").last().pixel_color

In [10]:
last_pixels.to_csv("top_position.csv")

# On trie les données

In [3]:
df2 = pd.read_csv("csv/top_position.csv")

## On trie les données pour qu'elles soient utilisable

In [4]:
df2.coordinate = df2.coordinate.str.split(pat=",")

In [5]:
df2["x"] = df2.coordinate.str[0]
df2["y"] = df2.coordinate.str[1]

In [6]:
df2["x"] = df2["x"].astype(int)
df2["y"] = df2["y"].astype(int)

## On enlève les colonnes ou les données sont inutilisables / ne servent ) rien
En effet, certaines colonnes sont incompletes, pas logique ce qui fait que je suis obligé de les enlever

In [7]:
df2.drop(df2[df2.coordinate.str.len() > 2].index, inplace=True)

In [8]:
df2.drop("coordinate", axis=1, inplace=True)

## On rajoute ensuite les pixels manquant
En effet, certains pixels manquent pour créer une matrice.
Ce tableau représente les différents **CHANGEMENT** de pixels. Certains pixels n'ont subi aucun changement, donc il faut les rajouter.

In [9]:
count = df2.groupby("x").y.count()
x_missing = count[count < 2000]

In [10]:
index_x = x_missing.index.tolist()

In [11]:
good = [i for i in range(0, 2000)]

In [12]:
list_append = []
for index in index_x:
    list_y = df2[df2.x == index].y.tolist()
    y_missing_list = set(good).difference(set(list_y))
    for y_missing in y_missing_list:
        list_append.append(["#FFFFFF", index, y_missing])

In [13]:
df_append = pd.DataFrame(list_append, columns=["pixel_color", "x","y"])

In [14]:
df3 = pd.concat([df2, df_append], ignore_index=True)

In [15]:
df3.sort_values(["x","y"], inplace=True)

In [26]:
df3.to_csv("final_pixels.csv")

# On génère l'image

In [19]:
df4 = pd.read_csv("csv/final_pixels.csv")

In [20]:
df4.set_index("Unnamed: 0", inplace=True)

In [21]:
import numpy as np
from PIL import Image
from PIL import ImageColor

In [22]:
df4.pixel_color = df4.pixel_color.apply(lambda r: list(ImageColor.getcolor(r, "RGB")))

KeyboardInterrupt: 

In [ ]:
array_image = df4.groupby("y").pixel_color.apply(list).tolist()

In [ ]:
numpy_array = np.array(array_image, dtype=np.uint8)

In [ ]:
image_final = Image.fromarray(numpy_array)

In [ ]:
cm = 1
image_final.save(f"export/final_x{cm:02}.jpg")

## On tente d'agrandir l'image

In [17]:
def multiply_array(list, int):
    final = []
    for element in list:
        for loop in range(int):
            final.append(element)
    return final

In [18]:
%%time
cm = 32
final_temp = []
for line in array_image:
    line_cm = multiply_array(line, cm)
    final_temp.append(line_cm)
array_image_cm = multiply_array(final_temp, cm)

NameError: name 'array_image' is not defined

In [ ]:
%%time
numpy_array = np.array(array_image_cm, dtype=np.uint8)

In [14]:
%%time
image_final = Image.fromarray(numpy_array)

CPU times: total: 1.52 s
Wall time: 1.51 s


In [15]:
%%time
image_final.save(f"export/final_x{cm:02}.jpg")

CPU times: total: 2.75 s
Wall time: 2.76 s
